In [ ]:
import os
import torch
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
from utils.graph import draw_confusion_matrix
from utils.configs import config
from utils.data_loader import get_test_loader
from utils.model_list import xception
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from openmax.openmax import compute_openmax
from openmax.openmax_utils import (
    get_openmax_predict_int,
    get_openmax_predict_bin,
    get_int_labels,
    get_bin_labels,
    compute_roc,
    compute_pr,
    plot_roc,
    plot_pr,
)

In [ ]:
device = torch.device('cuda:0')

In [ ]:
net = xception(config=config)

In [ ]:
net.load_state_dict(torch.load(f'model_weights/fold_{config.fold}/model.pth'))

In [ ]:
net.to(device)
net.eval()

In [ ]:
test_dl, class_map = get_test_loader(config)

In [ ]:
for k, v in class_map.items():
    class_map[k] = v.capitalize()

In [ ]:
for k in range(16, len(class_map)):
    class_map.pop(k, None)

class_map[16] = 'Unknown'

In [ ]:
openmax = []
prob_u = []
y_true = []

with torch.no_grad():
    for data in test_dl:
        x = data[0].to(device)
        labels = data[1].cpu().numpy()
        out = net(x).cpu().numpy()

        for logits, label in zip(out, labels):
            temp_openmax, temp_prob_u = compute_openmax(logits, config.fold)
            openmax.append(temp_openmax)
            prob_u.append(temp_prob_u)
            y_true.append(label)

In [ ]:
y_true = np.asarray(y_true)

In [ ]:
y_true_bin = get_bin_labels(y_true)
y_true_int = get_int_labels(y_true)

In [ ]:
roc = compute_roc(y_true_bin, prob_u)
pr = compute_pr(y_true_bin, prob_u)

In [ ]:
plot_roc(roc)

In [ ]:
plot_pr(pr)

In [ ]:
roc_thresholds = roc['thresholds']
pr_thresholds = pr['thresholds']

In [ ]:
best_idx = np.argmax(roc['tpr'] - roc['fpr'])
best_thresh = roc_thresholds[best_idx]

In [ ]:
# best_idx = np.argmax(pr['f1'])
# best_thresh = pr_thresholds[best_idx]

In [ ]:
def get_unique(arr):
    unique, counts = np.unique(arr, return_counts=True)
    print(dict(zip(unique, counts)))

In [ ]:
y_pred_bin = []
for probs in openmax:
    temp = get_openmax_predict_bin(probs, threshold=best_thresh)
    y_pred_bin.append(temp)
y_pred_bin = np.asarray(y_pred_bin)

In [ ]:
f1_bin = f1_score(y_true_bin, y_pred_bin)
pre_bin = precision_score(y_true_bin, y_pred_bin)
rec_bin = recall_score(y_true_bin, y_pred_bin)
acc_bin = accuracy_score(y_true_bin, y_pred_bin)

In [ ]:
diff = y_true_bin - y_pred_bin
get_unique(diff)

In [ ]:
get_unique(y_true_bin)
get_unique(y_pred_bin)

In [ ]:
y_pred_int = []
for probs in openmax:
    temp = get_openmax_predict_int(probs, threshold=best_thresh)
    y_pred_int.append(temp)
y_pred_int = np.asarray(y_pred_int)

In [ ]:
probs

In [ ]:
get_unique(y_true_int)
get_unique(y_pred_int)

In [ ]:
cm = confusion_matrix(y_true_int, y_pred_int, normalize='true')

In [ ]:
cm_df = pd.DataFrame(cm, index=class_map.values(), columns=class_map.values())
cm_df.to_csv(f'results/cm_fold_{config.fold}.csv')

In [ ]:
cm = np.round(cm, 2)

In [ ]:
draw_confusion_matrix(cm, class_map.values())

In [ ]:
f1_int_macro = f1_score(y_true_int, y_pred_int, average='macro')
pre_int_macro = precision_score(y_true_int, y_pred_int, average='macro')
rec_int_macro = recall_score(y_true_int, y_pred_int, average='macro')

f1_int_micro = f1_score(y_true_int, y_pred_int, average='micro')
pre_int_micro = precision_score(y_true_int, y_pred_int, average='micro')
rec_int_micro = recall_score(y_true_int, y_pred_int, average='micro')

acc_int = accuracy_score(y_true_int, y_pred_int)

In [ ]:
metrics = {}
metrics['auroc'] = roc['auroc']
metrics['aupr'] = pr['aupr']

metrics['f1_bin'] = f1_bin
metrics['pre_bin'] = pre_bin
metrics['rec_bin'] = rec_bin
metrics['acc_bin'] = acc_bin

metrics['f1_int_macro'] = f1_int_macro
metrics['pre_int_macro'] = pre_int_macro
metrics['rec_int_macro'] = rec_int_macro

metrics['f1_int_micro'] = f1_int_micro
metrics['pre_int_micro'] = pre_int_micro
metrics['rec_int_micro'] = rec_int_micro

metrics['acc_int'] = acc_int

In [ ]:
metrics

In [ ]:
with open(f'results/metrics_{config.fold}.txt', 'w') as file:
    for k, v in metrics.items():
        file.write(f'{k}: {v}\n')